In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
from pathlib import Path
import json
import random
import re
import uuid
import shutil
import os
from functools import partial
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

random.seed()

**Load vocaburay from tencent's word embedding file**

In [35]:
# with open('/home/shizai/qfeng/Tencent_AILab_ChineseEmbedding.txt', 'r', errors='ignore') as f:
#     tencent_words = [line.strip('\n').split(' ')[0] for line in f.readlines()]
    
# tencent_chars = list(set(c for w in tencent_words for c in w))
# len(tencent_words), len(tencent_chars)

(8825659, 22781)

In [36]:
# tencent_words[:10]

['8825658', '</s>', '的', '。', ',', '了', '、', '“', '”', '是']

In [37]:
# tencent_chars[:10]

['赐', '筂', '鳤', '糷', '炅', '笅', '鸡', '逹', '嵢', '梯']

**Load vocabury from jieba's dictionary file**

In [193]:
jieba_words = []

with open('/datadisk2/qfeng/jieba/extra_dict/dict.txt.small', 'r') as f:
    for line in f.readlines():
        word_unit = line.strip('\n').split(' ')        
        jieba_words.append(word_unit)

jieba_chars = list(set(c for w in jieba_words for c in w[0]))

In [194]:
len(jieba_words), len(jieba_chars)

(109750, 8939)

In [195]:
jieba_words[:10]

[['的', '3188252', 'uj'],
 ['了', '883634', 'ul'],
 ['是', '796991', 'v'],
 ['在', '727915', 'p'],
 ['和', '555815', 'c'],
 ['有', '423765', 'v'],
 ['他', '401339', 'r'],
 ['不', '360331', 'd'],
 ['我', '328841', 'r'],
 ['人', '313209', 'n']]

In [196]:
jieba_chars[:10]

['殂', '赐', '鎰', '姞', '物', '纨', '炅', '笅', '玁', '颔']

In [197]:
jieba_word_freq_sum = sum([int(w[1]) for w in jieba_words])
jieba_pure_words = [w[0] for w in jieba_words]
jieba_word_freq_pcts = [int(w[1])/word_freq_sum for w in jieba_words]

**Generate from jieba vocabulary**

In [ ]:
outp_path = '/datadisk2/qfeng/ocr_synthesis/jieba_texts'

In [202]:
import random
import bisect
import collections

def cdf(weights):
    total = sum(weights)
    result = []
    cumsum = 0
    for w in weights:
        cumsum += w
        result.append(cumsum / total)
    return result

def choice(population, weights):
    assert len(population) == len(weights)
    cdf_vals = cdf(weights)
    x = random.random()
    idx = bisect.bisect(cdf_vals, x)
    return population[idx]

In [208]:
choice(jieba_pure_words, jieba_word_freq_pcts)

'刘宗周'

In [228]:
# def gen_texts(words, freq_pcts, min_len, max_len, n_outp_words):
#     outp = []
#     while len(outp) < n_outp_words :
#         text = ''
#         while len(text) < min_len:
#             choiced = choice(words, freq_pcts).strip()
#             text = text + choiced
#         if len(text) <= max_len:
#             outp.append(text)
#     return outp

# gen_texts(jieba_pure_words, jieba_word_freq_pcts, min_len=5, max_len=5, n_outp_words=10)

In [246]:
def next_text(words, freq_pcts, min_len, max_len):
    while True:
        text = ''
        while len(text) < min_len:
            choiced = choice(words, freq_pcts).strip()
            text = text + choiced
        if len(text) <= max_len:
            yield text

In [247]:
def gen_texts(fid, gen_counts, gen_len, pure_words, word_freq_pcts):
    random.seed(random.randint(1,100))
    generator = next_text(pure_words, word_freq_pcts, min_len=gen_len, max_len=gen_len)
    with open(outp_path  + '/{}_lens/{}.txt'.format(gen_len, fid), 'w') as f:
        for _ in range(gen_counts):
            f.write(next(generator) + '\n')

In [248]:
# gen_texts(fid = 0, gen_counts=10, gen_len=5, pure_words=jieba_pure_words, word_freq_pcts=jieba_word_freq_pcts)

In [249]:
gen_counts = 10000
gen_batches = 200
for gen_len in [5, 10, 15]:
    with ProcessPoolExecutor(max_workers=40) as ex:
        func = partial(gen_texts, gen_counts=gen_counts, gen_len=gen_len,
                       pure_words=jieba_pure_words,
                       word_freq_pcts=jieba_word_freq_pcts)
        for i in range(gen_batches):
            ex.submit(func, i)

In [252]:
from pathlib import Path

word_counter = set()

for gen_len in [5, 10, 15]:
    for p in (Path(outp_path)/('{}_lens'.format(gen_len))).glob('*'):
        with open(str(p), 'r') as f:
            for line in f.readlines():
                word_counter.add(line.strip('\n'))
                
char_counter = {c for w in word_counter for c in w}

NameError: name 'outp_path' is not defined